In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


In [2]:
# Laden van de dataset
dataset = pd.read_csv('./CasusData.csv')  # Vervang 'jouw_bestandsnaam.csv' door de werkelijke bestandsnaam


# Data Preprocessing
### 1. Negeren van WAP's met een waarde van 100

In [3]:
# 1. Negeren van WAP's met een waarde van 100
# Maak een masker om rijen met WAP-waarden van 100 te negeren
mask = ~(dataset == 100).any(axis=1)

# Controleer of er voldoende gegevens overblijven na filtering
if mask.sum() == 0:
    print("Er zijn geen geldige samples over na filtering.")
else:
    # Pas het masker toe op de dataset
    dataset_filtered = dataset[mask]

    # Behandeling van ontbrekende waarden
    imputer = SimpleImputer(strategy='mean')
    dataset_imputed = pd.DataFrame(imputer.fit_transform(dataset_filtered), columns=dataset.columns)



Er zijn geen geldige samples over na filtering.


### 2. Behandeling van ontbrekende waarden

In [4]:
# 2. Behandeling van ontbrekende waarden
imputer = SimpleImputer(strategy='mean')
dataset_imputed = pd.DataFrame(imputer.fit_transform(dataset), columns=dataset.columns)

### 3. Feature scaling (normalisatie)

In [5]:
# 3. Feature scaling (normalisatie)
# Hier gebruiken we StandardScaler om de features te normaliseren.
scaler = StandardScaler()
features_scaled = scaler.fit_transform(dataset_imputed.iloc[:, :520])

### 4. K-means clustering voor groepering van locaties

In [6]:
# 4. K-means clustering voor groepering van locaties
kmeans = KMeans(n_clusters=3, random_state=42)
dataset['LOCATION_CLUSTER'] = kmeans.fit_predict(features_scaled)

### 5. Doelvariabele (WALKING_PATTERN)

In [7]:
# 5. Nieuwe kolom: WALKING_PATTERN (bijvoorbeeld op basis van locatieverandering)
dataset['WALKING_PATTERN'] = (dataset['LATITUDE'].diff() != 0) | (dataset['LONGITUDE'].diff() != 0)
dataset['WALKING_PATTERN'] = dataset['WALKING_PATTERN'].astype(int)


### 6. Groeperen op individuele gebruikers

In [8]:
# 6. Groeperen op individuele gebruikers
grouped_data = dataset.groupby('USERID').agg({
    'WALKING_PATTERN': 'max',  # Aggregeer het wandelpatroon (max waarde over tijd)
    'LOCATION_CLUSTER': 'max',  # Aggregeer de locatiecluster (max waarde over tijd)
    # Voeg andere gewenste aggregaties toe voor extra informatie
}).reset_index()

In [13]:
# Doelvariabele
target = grouped_data['WALKING_PATTERN']

# Features voor voorspelling (bijvoorbeeld locatiecluster)
features_grouped = grouped_data[['LOCATION_CLUSTER']]

In [10]:
# Split de data in trainings- en testsets
X_train, X_test, y_train, y_test = train_test_split(features_grouped, target, test_size=0.2, random_state=42)

# Train een Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
# Voorspel de WALKING_PATTERN op de testset
predictions = classifier.predict(X_test)

# Evaluatie van het model
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


### Eenvoudige analyse

In [15]:
# Eenvoudige evaluatie
print('Classification Report:\n', classification_report(y_test, predictions))

Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

